### Lazy Propagation
- Segment Tree에서 노드의 업데이트를 미루다가, 쿼리를 받으면 그때 업데이트를 하는 방식
- 각 노드에 대응하는 lazy 배열을 두어, 업데이트를 미룬다.
  - 해당 노드가 관장하는 구간 전체에 대해서 어떤 연산을 미루고 있는지에 대한 정보가 들어있다.
    - 구간합 세그의 경우 얼마를 더할 것인지, xor 세그의 경우 부호를 바꿀지의 여부 등
- 쿼리를 받으면, 해당 노드의 lazy를 업데이트하고, 자식 노드에게 propagate한다.
- 길이 $N$인 구간 $[a, b]$의 모든 인덱스에 연산을 적용하는 쿼리 $Q$ 의 구현을 Naive하게 적용한다면 그 구간의 길이 $b-a = K$ 라고 했을 때 $\Omicron(K \log N)$ 이 걸린다.\
하지만 Lazy Propagation을 적용하면 $\Omicron(\log N)$ 으로 줄일 수 있다.

#### Propagate
- 완전히 포함되면 lazy만 매기고, propagate한다

### 구현(재귀)
- query의 내부를 수정하기 쉬운 템플릿
- 코드 주석은 10999(구간 합 구하기 2) 참고

In [ ]:
class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    self.L[i] += (e - s) * self.lazy[i]
    if i < self.size :
      self.lazy[i*2] += self.lazy[i]
      self.lazy[i*2+1] += self.lazy[i]
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def update(self, l, r, x):
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid)
    self._update(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

### iterative 구현
- L을 그대로 입력받는 것엔 잘 안된다. 이유를 모르겠음

In [ ]:
class LazySegmentTree:
  def __init__(self, L, func, default=0):
    self._default = default
    self._func = func
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()
    self.lazy = [0] * (2 * _size)
    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = func(self.L[i + i], self.L[i + i + 1])

  def _push(self, idx) : #해당 idx에 포함되는 자식들에게 쿼리를 전파한다 
    q, self.lazy[idx] = self.lazy[idx], 0

    self.lazy[2 * idx] += q
    self.lazy[2 * idx + 1] += q
    self.L[2 * idx] += q
    self.L[2 * idx + 1] += q

  def _update(self, idx): #노드의 idx를 업데이트해서 조상들에게 적용돼있는 모든 쿼리를 알 수 있게 한다
    for i in reversed(range(1, idx.bit_length())):
      self._push(idx >> i)

  def _build(self, idx): #해당 idx에 대한 변경사항을 조상들에게 알림
    idx >>= 1
    while idx:
      self.L[idx] = self._func(self.L[2 * idx], self.L[2 * idx + 1]) + self.lazy[idx]
      idx >>= 1

  def add(self, s, e, v): #add v on [s, e)
    s = _s = s + self._size
    e = _e = e + self._size
    while s < e:
      if s & 1:
        self.lazy[s] += v
        self.L[s] += v
        s += 1
      if e & 1: 
        e -= 1
        self.lazy[e] += v
        self.L[e] += v
      s >>= 1
      e >>= 1

    # 업데이트 될 구간의 조상들에게 업데이트가 됐음을 알린다
    self._build(_s)
    self._build(_e - 1)

  def query(self, s, e, default=0): #query on [s, e)
    s += self._size
    e += self._size

    # lazy에 저장돼있는 쿼리를 모두 적용한다
    self._update(s)
    self._update(e - 1)

    res = default
    while s < e:
      if s & 1:
        res = self._func(res, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        res = self._func(res, self.L[e])
      s >>= 1
      e >>= 1
    return res